In [2]:
!pwd

/workspace/Paddle/paddle_build/notebook


In [12]:
wenetspeech_conformer_model='http://mobvoi-speech-public.ufile.ucloud.cn/public/wenet/wenetspeech/20211025_conformer_exp.tar.gz'

In [14]:
!wget $wenetspeech_conformer_model

--2021-11-18 03:15:28--  http://mobvoi-speech-public.ufile.ucloud.cn/public/wenet/wenetspeech/20211025_conformer_exp.tar.gz
Connecting to 172.19.56.199:3128... connected.
Proxy request sent, awaiting response... 200 OK
Length: 478183697 (456M) [application/x-gzip]
Saving to: '20211025_conformer_exp.tar.gz'

20211025_conformer_ 100%[===================>] 456.03M  3.52MB/s    in 2m 2s   

2021-11-18 03:17:56 (3.73 MB/s) - '20211025_conformer_exp.tar.gz' saved [478183697/478183697]



In [15]:
!tar zxvf '20211025_conformer_exp.tar.gz'

20211025_conformer_exp/
20211025_conformer_exp/global_cmvn
20211025_conformer_exp/words.txt
20211025_conformer_exp/final.pt
20211025_conformer_exp/train.yaml


In [37]:
import logging
import os
import random
import sys
import numpy as np
from pprint import pprint
import paddle
import yaml

In [27]:
conf_path='20211025_conformer_exp/train.yaml'
model_path='20211025_conformer_exp/final.pt'
cmvn_path='20211025_conformer_exp/global_cmvn'
vocab_path='20211025_conformer_exp/words.txt'

In [28]:
!wc -l $vocab_path

5537 20211025_conformer_exp/words.txt


In [35]:
conf = yaml.load(open(conf_path), yaml.FullLoader)
pprint(conf)

{'accum_grad': 16,
 'cmvn_file': 'exp/conformer/global_cmvn',
 'dataset_conf': {'batch_conf': {'batch_size': 32, 'batch_type': 'static'},
                  'fbank_conf': {'dither': 1.0,
                                 'frame_length': 25,
                                 'frame_shift': 10,
                                 'num_mel_bins': 80},
                  'filter_conf': {'max_length': 1200,
                                  'min_length': 10,
                                  'token_max_length': 100,
                                  'token_min_length': 1},
                  'resample_conf': {'resample_rate': 16000},
                  'shuffle': True,
                  'shuffle_conf': {'shuffle_size': 1500},
                  'sort': True,
                  'sort_conf': {'sort_size': 1000},
                  'spec_aug': True,
                  'spec_aug_conf': {'max_f': 30,
                                    'max_t': 50,
                                    'num_f_mask': 2,
       

# load model & dump to paddle

In [39]:
import torch

In [61]:
model_state_dict = torch.load(model_path, map_location=lambda storage, loc: storage)

In [62]:
param_cnt=0
for key, val in model_state_dict.items():
    print(key, "\t", val.shape, "\t", val.dtype)
    param_cnt += 1

encoder.global_cmvn.mean 	 torch.Size([80]) 	 torch.float32
encoder.global_cmvn.istd 	 torch.Size([80]) 	 torch.float32
encoder.embed.conv.0.weight 	 torch.Size([512, 1, 3, 3]) 	 torch.float32
encoder.embed.conv.0.bias 	 torch.Size([512]) 	 torch.float32
encoder.embed.conv.2.weight 	 torch.Size([512, 512, 3, 3]) 	 torch.float32
encoder.embed.conv.2.bias 	 torch.Size([512]) 	 torch.float32
encoder.embed.out.0.weight 	 torch.Size([512, 9728]) 	 torch.float32
encoder.embed.out.0.bias 	 torch.Size([512]) 	 torch.float32
encoder.after_norm.weight 	 torch.Size([512]) 	 torch.float32
encoder.after_norm.bias 	 torch.Size([512]) 	 torch.float32
encoder.encoders.0.self_attn.pos_bias_u 	 torch.Size([8, 64]) 	 torch.float32
encoder.encoders.0.self_attn.pos_bias_v 	 torch.Size([8, 64]) 	 torch.float32
encoder.encoders.0.self_attn.linear_q.weight 	 torch.Size([512, 512]) 	 torch.float32
encoder.encoders.0.self_attn.linear_q.bias 	 torch.Size([512]) 	 torch.float32
encoder.encoders.0.self_attn.linear

encoder.encoders.11.conv_module.depthwise_conv.bias 	 torch.Size([512]) 	 torch.float32
encoder.encoders.11.conv_module.norm.weight 	 torch.Size([512]) 	 torch.float32
encoder.encoders.11.conv_module.norm.bias 	 torch.Size([512]) 	 torch.float32
encoder.encoders.11.conv_module.pointwise_conv2.weight 	 torch.Size([512, 512, 1]) 	 torch.float32
encoder.encoders.11.conv_module.pointwise_conv2.bias 	 torch.Size([512]) 	 torch.float32
encoder.encoders.11.norm_ff.weight 	 torch.Size([512]) 	 torch.float32
encoder.encoders.11.norm_ff.bias 	 torch.Size([512]) 	 torch.float32
encoder.encoders.11.norm_mha.weight 	 torch.Size([512]) 	 torch.float32
encoder.encoders.11.norm_mha.bias 	 torch.Size([512]) 	 torch.float32
encoder.encoders.11.norm_ff_macaron.weight 	 torch.Size([512]) 	 torch.float32
encoder.encoders.11.norm_ff_macaron.bias 	 torch.Size([512]) 	 torch.float32
encoder.encoders.11.norm_conv.weight 	 torch.Size([512]) 	 torch.float32
encoder.encoders.11.norm_conv.bias 	 torch.Size([512]) 

In [63]:
print(param_cnt)

665


In [64]:
# dump torch model encoder to paddle encoder
def torch_paddle_param(model_state_dict):
    paddle_state_dict = {}
    for n, p in model_state_dict.items():
        print(f'-> name: {n} dim: {p.ndim}')
        
        # change norm.mean and norm variance
        name_change=True
        if 'norm.running_mean' in n:
            new_n = n.replace('norm.running_', 'norm._')
        elif 'norm.running_var' in n:
            new_n = n.replace('norm.running_var', 'norm._variance')
        else:
            name_change=False
            new_n = n
        if name_change:
            print(f"\t norm mean/var: {n} -> {new_n}")

        p = p.cpu().detach().numpy()
        
        # weight which is rank 2, transpose it
        if n.endswith('weight') and p.ndim == 2:
            new_p = p.T
            print(f"\t liner transpose: {n}: {p.shape} -> {new_p.shape}")
        else:
            new_p = p
        
        # text embedding layer
        if 'decoder.embed.0.weight' in n:
            new_p = p

        if 'global_cmvn.mean' in n:
            print("\t cmvn:", p, p.dtype)
        if 'global_cmvn.istd' in n:
            print("\t istd:", p, p.dtype)

        paddle_state_dict[new_n] = new_p
    return paddle_state_dict

In [65]:
paddle_model_state_dict = torch_paddle_param(model_state_dict)

-> name: encoder.global_cmvn.mean dim: 1
	 cmvn: [11.837256 12.473205 13.416769 14.07741  14.692714 15.134645 15.425054
 15.520304 15.66498  15.682886 15.831343 15.901057 16.043106 16.141926
 16.14606  16.172684 16.13231  16.065538 16.170683 15.998217 15.867838
 16.081026 15.909136 16.032068 15.948578 16.035398 15.919972 16.01213
 15.935732 15.914797 15.949415 15.914241 15.920595 15.979177 15.986891
 16.046032 16.110855 16.11682  16.129877 16.085758 16.134706 16.098186
 16.202894 16.195677 16.265987 16.368603 16.485243 16.530725 16.586134
 16.682056 16.643587 16.623293 16.638262 16.703993 16.758455 16.818436
 16.887299 16.890385 16.816685 16.731005 16.674946 16.562819 16.506945
 16.427153 16.336958 16.224352 16.122593 16.074572 16.045864 15.997705
 15.955503 15.925531 15.884871 15.847951 15.812487 15.79125  15.698866
 15.451056 15.043111 14.453489] float32
-> name: encoder.global_cmvn.istd dim: 1
	 istd: [0.33298537 0.3044571  0.29546332 0.30167487 0.30146667 0.2972203
 0.29171112 0.28

In [71]:
eq = []
for key, val in model_state_dict.items():
    if 'decoder.embed.0.weight' in key:
        # text embeding
        val = val
    elif key.endswith('weight') and val.ndim == 2:
        # linear weight
        val = val.T  
    eq.append(np.allclose(val, paddle_model_state_dict[key]))         
print("all param equal:", np.array(eq).all())

all param equal: True


## save paddle model

In [73]:
paddle.save(paddle_model_state_dict, "wenetspeech.npz")

/workspace/DeepSpeech-2.x/tools/venv/lib/python3.7/site-packages/paddle/framework/io.py:415: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  if isinstance(obj, collections.Iterable) and not isinstance(obj, (


In [74]:
import numpy 

In [77]:
a = np.load("wenetspeech.npz", allow_pickle=True)

In [79]:
print(a)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [81]:
print(a.keys())

dict_keys(['encoder.global_cmvn.mean', 'encoder.global_cmvn.istd', 'encoder.embed.conv.0.weight', 'encoder.embed.conv.0.bias', 'encoder.embed.conv.2.weight', 'encoder.embed.conv.2.bias', 'encoder.embed.out.0.weight', 'encoder.embed.out.0.bias', 'encoder.after_norm.weight', 'encoder.after_norm.bias', 'encoder.encoders.0.self_attn.pos_bias_u', 'encoder.encoders.0.self_attn.pos_bias_v', 'encoder.encoders.0.self_attn.linear_q.weight', 'encoder.encoders.0.self_attn.linear_q.bias', 'encoder.encoders.0.self_attn.linear_k.weight', 'encoder.encoders.0.self_attn.linear_k.bias', 'encoder.encoders.0.self_attn.linear_v.weight', 'encoder.encoders.0.self_attn.linear_v.bias', 'encoder.encoders.0.self_attn.linear_out.weight', 'encoder.encoders.0.self_attn.linear_out.bias', 'encoder.encoders.0.self_attn.linear_pos.weight', 'encoder.encoders.0.feed_forward.w_1.weight', 'encoder.encoders.0.feed_forward.w_1.bias', 'encoder.encoders.0.feed_forward.w_2.weight', 'encoder.encoders.0.feed_forward.w_2.bias', 'en